In [90]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from matplotlib import pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [91]:
model = nn.Sequential(

    # Block 1
    nn.Conv2d(3, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2),      # 32×32 → 16×16

    # Block 2
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2),      # 16×16 → 8×8

    # Block 3
    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(2),      # 8×8 → 4×4

    # Block 4
    nn.Conv2d(128, 256, kernel_size=3, padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(),

    nn.Flatten(),

    nn.Linear(256 * 4 * 4, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 100)   # CIFAR-100
)


In [92]:
def make_model():
    return nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2),      # 32×32 → 16×16

        # Block 2
        nn.Conv2d(32, 64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2),      # 16×16 → 8×8

        # Block 3
        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2),      # 8×8 → 4×4

        # Block 4
        nn.Conv2d(128, 256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),

        nn.Flatten(),

        nn.Linear(256 * 4 * 4, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 100)   # CIFAR-100
    )


In [93]:
def set_seed(seed):
    import random, numpy as np, torch
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [94]:
# 3. Προετοιμασία συνόλου δεδομένων CIFAR-100

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])


train_dataset = datasets.CIFAR100(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR100(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [95]:
# 4. Ορισμός συναρτήσεων κόστους και βελτιστοποίησης
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [96]:
# --------------------------------------------------------
# 5. Εκπαίδευση
# --------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = make_model().to(device)

def train_one(model, train_loader, test_loader, criterion, optimizer, device, epochs=10):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        correct, total = 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs,1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = 100*correct/total
        val_acc = evaluate(model, test_loader, device)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Val Acc: {val_acc:.2f}%")




In [97]:
def evaluate(model, loader, device):
    model.eval()
    correct, total = 0,0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs,1)
            correct += (preds==labels).sum().item()
            total += labels.size(0)
    return 100*correct/total

In [98]:
lrs = [1e-2, 1e-3, 1e-4, 1e-5]
for lr in lrs:
    print(f"Training with LR={lr}")
    set_seed(0)
    model = make_model()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    train_one(model, train_loader, test_loader, criterion, optimizer, device, epochs=5)


Training with LR=0.01
Epoch 1/5, Loss: 4.6903, Val Acc: 1.00%
Epoch 2/5, Loss: 4.6089, Val Acc: 1.00%
Epoch 3/5, Loss: 4.6091, Val Acc: 1.00%
Epoch 4/5, Loss: 4.6092, Val Acc: 1.00%
Epoch 5/5, Loss: 4.6089, Val Acc: 1.00%
Training with LR=0.001
Epoch 1/5, Loss: 3.8562, Val Acc: 18.94%
Epoch 2/5, Loss: 3.2595, Val Acc: 28.26%
Epoch 3/5, Loss: 2.9269, Val Acc: 33.73%
Epoch 4/5, Loss: 2.6949, Val Acc: 38.77%
Epoch 5/5, Loss: 2.5147, Val Acc: 40.52%
Training with LR=0.0001
Epoch 1/5, Loss: 3.7486, Val Acc: 26.81%
Epoch 2/5, Loss: 2.9680, Val Acc: 35.31%
Epoch 3/5, Loss: 2.5909, Val Acc: 37.54%
Epoch 4/5, Loss: 2.3492, Val Acc: 41.47%
Epoch 5/5, Loss: 2.1572, Val Acc: 43.80%
Training with LR=1e-05
Epoch 1/5, Loss: 4.3883, Val Acc: 10.89%
Epoch 2/5, Loss: 3.9991, Val Acc: 17.01%
Epoch 3/5, Loss: 3.7439, Val Acc: 20.41%
Epoch 4/5, Loss: 3.5634, Val Acc: 23.74%
Epoch 5/5, Loss: 3.4105, Val Acc: 26.02%


In [100]:
optimizers = [optim.Adam, optim.SGD, optim.RMSprop]
for opt_cls in optimizers:
    print(f"Training with {opt_cls.__name__}")
    set_seed(0)
    model = make_model().to(device)
    optimizer = opt_cls(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    train_one(model, train_loader, test_loader, criterion, optimizer, device, epochs=5)


Training with Adam
Epoch 1/5, Loss: 3.8562, Val Acc: 18.94%
Epoch 2/5, Loss: 3.2595, Val Acc: 28.26%
Epoch 3/5, Loss: 2.9269, Val Acc: 33.73%
Epoch 4/5, Loss: 2.6949, Val Acc: 38.77%
Epoch 5/5, Loss: 2.5147, Val Acc: 40.52%
Training with SGD
Epoch 1/5, Loss: 4.5429, Val Acc: 4.85%
Epoch 2/5, Loss: 4.3732, Val Acc: 7.94%
Epoch 3/5, Loss: 4.2136, Val Acc: 11.32%
Epoch 4/5, Loss: 4.0762, Val Acc: 13.67%
Epoch 5/5, Loss: 3.9437, Val Acc: 15.66%
Training with RMSprop
Epoch 1/5, Loss: 4.4432, Val Acc: 7.23%
Epoch 2/5, Loss: 4.1149, Val Acc: 10.21%
Epoch 3/5, Loss: 3.8587, Val Acc: 13.74%
Epoch 4/5, Loss: 3.5949, Val Acc: 20.31%
Epoch 5/5, Loss: 3.2931, Val Acc: 24.77%


In [ ]:
losses = [nn.CrossEntropyLoss(), nn.MSELoss()]
for loss_fn in losses:
    print(f"Training with {loss_fn}")
    set_seed(0)
    model = make_model().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    train_one(model, train_loader, test_loader, loss_fn, optimizer, device, epochs=5)
